In [80]:
import logging
import os
import sys
import shutil
import tempfile
import nibabel as nib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split 
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from models.resnet import resnet18 

import monai
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import DataLoader, ImageDataset
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    RandRotate90,
    Resize,
    ScaleIntensity,
)


pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# print_config()



#### **1. ResNet18: El Caballo de Batalla Generalista**

**ResNet18** es una arquitectura de red neuronal convolucional muy popular, parte de la familia Residual Networks (ResNet). Su innovación principal radica en el uso de **"bloques residuales"** o "saltos" (skip connections). Estos saltos permiten que la información y los gradientes fluyan más fácilmente a través de muchas capas, resolviendo el problema del "gradiente desvanecido" que dificultaba el entrenamiento de redes muy profundas.

* **Pre-entrenamiento:** Comúnmente, ResNet18 se pre-entrena en el dataset **ImageNet**. Este es un gigantesco conjunto de datos con millones de imágenes de objetos cotidianos (perros, coches, sillas, etc.) de 1000 categorías diferentes.
* **Ventajas en imágenes médicas:** A pesar de haber sido entrenado con imágenes naturales, las características de bajo nivel que ResNet18 aprende de ImageNet (detección de bordes, patrones de textura) son sorprendentemente útiles como punto de partida para el análisis de imágenes médicas. Es un excelente punto de inicio general.
* **Consideración:** Puede haber una "brecha de dominio" entre las imágenes de ImageNet y las imágenes médicas, lo que significa que el fine-tuning es crucial para adaptar el modelo a las particularidades de los datos médicos (contrastes, resoluciones, tipos de ruido).

---

#### **2. MedicalNet: Especialización para el Dominio Médico**

**MedicalNet** es una iniciativa que proporciona modelos (incluyendo variantes de ResNet como ResNet18) que han sido **pre-entrenados específicamente en un vasto y diverso conjunto de datos de imágenes médicas**.

* **Pre-entrenamiento:** A diferencia de ResNet18 estándar, MedicalNet ha sido entrenado con millones de imágenes provenientes de diversas modalidades médicas (MRI, CT, rayos X, ultrasonido) y cubriendo diferentes órganos y patologías.
* **Ventajas en imágenes médicas:**
    * **Mayor relevancia de las características:** Al estar pre-entrenado en datos médicos, MedicalNet ya ha aprendido patrones y características que son intrínsecamente más relevantes para el diagnóstico y análisis clínico.
    * **Menor brecha de dominio:** Esto puede traducirse en un mejor rendimiento inicial, una convergencia más rápida durante el fine-tuning y, potencialmente, un mejor rendimiento final con menos datos de entrenamiento específicos para tu tarea.
    * **Adaptado a diferentes modalidades:** Su entrenamiento diverso lo hace robusto para trabajar con distintos tipos de imágenes médicas.



In [69]:
# Dimensiones de tus volúmenes de MRI usamos 256x256x256 ya que es un tamaño común para imágenes de resonancia magnética cerebral en freesurfer
input_D = 256
input_H = 256
input_W = 256

# Número de canales (1 para norm.mgz)
input_C = 1 
model = resnet18(sample_input_D=input_D,
                 sample_input_H=input_H,
                 sample_input_W=input_W,
                 num_seg_classes=1)  # ATENTO A ESTO PORQUE ES IMPORTANTE PARA LA CLASIFICACION

/home/nicolas/Documentos/gliara/models/resnet.py:173: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


Pesos de MedicalNet y ResNet18:
https://share.weiyun.com/55sZyIx 

In [72]:
pretrained_weights_path = "pretrain/resnet_18_23dataset.pth"

try:
    state_dict = torch.load(pretrained_weights_path)
    model.load_state_dict(state_dict, strict=False)
    print(f"Pesos de ResNet-18 cargados exitosamente desde {pretrained_weights_path}")

except FileNotFoundError:
    print(f"Error: El archivo de pesos no se encontró en {pretrained_weights_path}")
    print("Asegúrate de que la ruta sea correcta y el archivo exista.")
except Exception as e:
    print(f"Ocurrió un error al cargar los pesos: {e}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Modelo movido a: {device}")

/tmp/ipykernel_61891/2331833451.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(pretrained_weights_path)


Pesos de ResNet-18 cargados exitosamente desde pretrain/resnet_18_23dataset.pth
Modelo movido a: cuda


In [73]:
# Define el número de clases para tu tarea de clasificación binaria (Alzheimer sí/no)
your_num_classes = 2
final_conv_layer = model.conv_seg[0]
num_in_features = final_conv_layer.in_channels
model.conv_seg = nn.Sequential(
    nn.Conv3d(num_in_features, your_num_classes,
              kernel_size=final_conv_layer.kernel_size,
              stride=final_conv_layer.stride,
              padding=final_conv_layer.padding)
)

print(f"Capa de salida (conv_seg) adaptada a {your_num_classes} clases para el proyecto Gliara.")

# Mueve el modelo completo (incluida la nueva capa) a la GPU
model.to(device)
print(f"Modelo movido a: {device}")


Capa de salida (conv_seg) adaptada a 2 clases para el proyecto Gliara.
Modelo movido a: cuda


# Preparar la data

In [60]:
DATA = pd.read_csv("AUMENTED_DATA.csv")
DATA["CDR"] = np.where(DATA["CDR"] != 0, 1, DATA["CDR"])
DATA.to_csv("AUMENTED_DATA2.csv", index=False)

In [74]:
DATA

,ID,CDR
0,OAS1_0001_MR1,0.0
1,OAS1_0002_MR1,0.0
2,OAS1_0010_MR1,0.0
3,OAS1_0011_MR1,0.0
4,OAS1_0013_MR1,0.0
...,...,...
397,A-OAS1_0210_MR1,1.0
398,A-OAS1_0240_MR1,1.0
399,A-OAS1_0262_MR1,0.0
400,A-OAS1_0206_MR1,0.0


In [75]:
BASE_DATA_DIR = "OASIS/1/processed"
df_gliara = DATA.rename(columns={'CDR': 'clasificacion'})
df_gliara['ubicacion'] = df_gliara['ID'].apply(
    lambda x: os.path.join(BASE_DATA_DIR, f"{x}.nii.gz")
)

df_gliara['clasificacion'] = df_gliara['clasificacion'].astype(int)


missing_files = []
for index, row in df_gliara.iterrows():
    if not os.path.exists(row['ubicacion']):
        missing_files.append(row['ubicacion'])

if missing_files:
    print(f"\nAdvertencia: {len(missing_files)} archivos .nii.gz no encontrados en {BASE_DATA_DIR}:")
    for mf in missing_files[:5]: # Muestra los primeros 5 archivos faltantes
        print(f"- {mf}")
    if len(missing_files) > 5:
        print(f"- y {len(missing_files) - 5} más...")
    print("Por favor, asegúrate de que 'BASE_DATA_DIR' y los 'ID' en tu DataFrame sean correctos.")
    # Opcional: Eliminar las filas de archivos no encontrados para evitar errores en el DataLoader
    df_gliara = df_gliara[~df_gliara['ubicacion'].isin(missing_files)]
    print(f"Se eliminaron {len(missing_files)} filas de archivos no encontrados del DataFrame.")

# Mezclar el DataFrame para asegurar aleatoriedad antes de la partición
df_gliara = df_gliara.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\nDataFrame 'df_gliara' listo con {len(df_gliara)} entradas:")
print(df_gliara.head())
print(f"Conteo de clases:\n{df_gliara['clasificacion'].value_counts()}")


Advertencia: 68 archivos .nii.gz no encontrados en OASIS/1/processed:
- OASIS/1/processed/OAS1_0110_MR1.nii.gz
- OASIS/1/processed/OAS1_0317_MR1.nii.gz
- OASIS/1/processed/OAS1_0337_MR1.nii.gz
- OASIS/1/processed/OAS1_0338_MR1.nii.gz
- OASIS/1/processed/OAS1_0341_MR1.nii.gz
- y 63 más...
Por favor, asegúrate de que 'BASE_DATA_DIR' y los 'ID' en tu DataFrame sean correctos.
Se eliminaron 68 filas de archivos no encontrados del DataFrame.

DataFrame 'df_gliara' listo con 334 entradas:
                ID  clasificacion                                 ubicacion
0    OAS1_0078_MR1              0    OASIS/1/processed/OAS1_0078_MR1.nii.gz
1  A-OAS1_0298_MR1              1  OASIS/1/processed/A-OAS1_0298_MR1.nii.gz
2    OAS1_0255_MR1              0    OASIS/1/processed/OAS1_0255_MR1.nii.gz
3  A-OAS1_0114_MR1              0  OASIS/1/processed/A-OAS1_0114_MR1.nii.gz
4    OAS1_0203_MR1              0    OASIS/1/processed/OAS1_0203_MR1.nii.gz
Conteo de clases:
clasificacion
0    194
1    140
Name:

In [76]:
train_ratio = 0.70
val_ratio = 0.15
test_ratio = 0.15

df_train, df_temp = train_test_split(
    df_gliara,
    test_size=(val_ratio + test_ratio), 
    stratify=df_gliara['clasificacion'], 
    random_state=42 
)


val_split_ratio = val_ratio / (val_ratio + test_ratio)

df_val, df_test = train_test_split(
    df_temp,
    test_size=test_ratio / (val_ratio + test_ratio), 
    stratify=df_temp['clasificacion'], 
    random_state=42 
)

print(f"\nPartición de datos completada:")
print(f"Tamaño del conjunto de Entrenamiento: {len(df_train)} imágenes")
print(f"Tamaño del conjunto de Validación: {len(df_val)} imágenes")
print(f"Tamaño del conjunto de Prueba: {len(df_test)} imágenes")

print("\nPrimeras 5 filas del DataFrame de Entrenamiento:")
print(df_train.head())

print("\nConteo de clases en el conjunto de Entrenamiento:")
print(df_train['clasificacion'].value_counts())

print("\nConteo de clases en el conjunto de Validación:")
print(df_val['clasificacion'].value_counts())

print("\nConteo de clases en el conjunto de Prueba:")
print(df_test['clasificacion'].value_counts())


Partición de datos completada:
Tamaño del conjunto de Entrenamiento: 233 imágenes
Tamaño del conjunto de Validación: 50 imágenes
Tamaño del conjunto de Prueba: 51 imágenes

Primeras 5 filas del DataFrame de Entrenamiento:
                  ID  clasificacion                                 ubicacion
118    OAS1_0228_MR1              0    OASIS/1/processed/OAS1_0228_MR1.nii.gz
22   A-OAS1_0208_MR1              0  OASIS/1/processed/A-OAS1_0208_MR1.nii.gz
181    OAS1_0060_MR1              1    OASIS/1/processed/OAS1_0060_MR1.nii.gz
292  A-OAS1_0169_MR1              0  OASIS/1/processed/A-OAS1_0169_MR1.nii.gz
287    OAS1_0114_MR1              0    OASIS/1/processed/OAS1_0114_MR1.nii.gz

Conteo de clases en el conjunto de Entrenamiento:
clasificacion
0    135
1     98
Name: count, dtype: int64

Conteo de clases en el conjunto de Validación:
clasificacion
0    29
1    21
Name: count, dtype: int64

Conteo de clases en el conjunto de Prueba:
clasificacion
0    30
1    21
Name: count, dtype: in

In [82]:
class MRIDataset(Dataset): # Asumiendo que Dataset ya fue importado de torch.utils.data
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

        # Las rutas y etiquetas ya están en el DataFrame
        self.image_paths = dataframe['ubicacion'].tolist()
        self.labels = dataframe['clasificacion'].tolist()

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # Cargar la imagen .nii.gz
        img = nib.load(img_path)
        data = img.get_fdata().astype(np.float32)

        # --- Normalización de intensidad ---
        min_val = np.min(data)
        max_val = np.max(data)
        if (max_val - min_val) > 1e-8:
            data = (data - min_val) / (max_val - min_val)
        else:
            data = np.zeros_like(data)

        # Añadir la dimensión del canal (C, D, H, W)
        data = np.expand_dims(data, axis=0) # Ahora es (1, 256, 256, 256)

        if self.transform:
            pass # Aquí irían tus transformaciones 3D

        image_tensor = torch.from_numpy(data)
        label_tensor = torch.tensor(label, dtype=torch.long)

        return image_tensor, label_tensor

# --- Creación de instancias de Dataset y DataLoader con los DataFrames particionados ---
# Asegúrate de que BATCH_SIZE y NUM_WORKERS estén definidos en tu script principal
BATCH_SIZE = 1 # Ajusta según la memoria de tu GPU
NUM_WORKERS = 4 # Ajusta según tu CPU

train_dataset = MRIDataset(dataframe=df_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

val_dataset = MRIDataset(dataframe=df_val)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

test_dataset = MRIDataset(dataframe=df_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

print("\nDataLoaders listos para el entrenamiento y evaluación de Gliara.")


DataLoaders listos para el entrenamiento y evaluación de Gliara.


# Entrenamiento

In [83]:
criterion = nn.CrossEntropyLoss()
LEARNING_RATE = 1e-4
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

NUM_EPOCHS = 30

/home/nicolas/miniconda3/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
best_val_loss = float('inf')
best_val_accuracy = 0.0

print(f"\n--- Comenzando el Fine-Tuning de MedicalNet ResNet-18 para Gliara ({NUM_EPOCHS} épocas) ---")

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_train_loss = running_loss / len(train_dataset)
    epoch_train_accuracy = correct_predictions / total_samples

    print(f"Época {epoch+1}/{NUM_EPOCHS} - Entrenamiento | Pérdida: {epoch_train_loss:.4f} | Precisión: {epoch_train_accuracy:.4f}")


--- Comenzando el Fine-Tuning de MedicalNet ResNet-18 para Gliara (30 épocas) ---


OutOfMemoryError: CUDA out of memory. Tried to allocate 512.00 MiB. GPU 0 has a total capacity of 1.95 GiB of which 429.62 MiB is free. Including non-PyTorch memory, this process has 1.52 GiB memory in use. Of the allocated memory 1.44 GiB is allocated by PyTorch, and 48.04 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)